Pair Programming ETL Transformación I

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:
 1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.
 2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

In [2]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:
 - Cargaremos los dos ficheros de datos
 - Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer: USA, Australia, New Zealand, South Africa, Papua New Guinea

In [3]:
# CARGAMOS LOS DATOS
df_attacks = pd.read_csv('/mnt/d/Adalab/ejercicios-de-adalab/modulo-2/8-ETL-mod-2/files/attacks_limpieza_completa.csv', index_col = 0)
df_etl1 = pd.read_csv('/mnt/d/Adalab/ejercicios-de-adalab/modulo-2/8-ETL-mod-2/files/df_ETL1_clima.csv')

In [ ]:
# COMPROBAMOS LOS NOMBRES Y FORMATOS PARA EXTRAER LOS PAÍSES QUE NOS INTERESAN
df_attacks['country'].unique()

In [5]:
# EXTRAEMOS 
df_paises = df_attacks[df_attacks['country'].isin(['usa', 'australia', 'new zealand', 'papua new guinea'])]
df_paises['country'].unique()

array(['usa', 'australia', 'new zealand', 'papua new guinea'],
      dtype=object)

 - Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [6]:
df_etl1.head(3)

,Unnamed: 0,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,USA
1,1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,8,1014,none,0,0,180,3,USA
2,2,9,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",3,15,7,1012,none,0,0,165,3,USA


In [ ]:
# RESETEAR EL INDEX DEL FICHERO DE AYER PARA QUE NO SE REPITAN LOS NOMBRES DE LAS COLUMNAS 
df_etl1.reset_index()

Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
 - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
 - Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [8]:
# COMO SUGERIDO, UTILIZAMOS LA LIBRERIA 'AST' DE PYTHON QUE MO PERMITE CASTEAR
# UN STRING COMPUESTO POR DICCIONARIOS, LISTAS O TUPLAS. EL OBJETIVO
# ES QUEDARNOS CON LISTAS EN LA COLUMNA
df_etl1['wind_profile'][2] 

"[{'layer': '950mb', 'direction': -9999, 'speed': -9999}, {'layer': '900mb', 'direction': 185, 'speed': 4}, {'layer': '850mb', 'direction': 200, 'speed': 5}, {'layer': '800mb', 'direction': 210, 'speed': 5}, {'layer': '750mb', 'direction': 225, 'speed': 5}, {'layer': '700mb', 'direction': 245, 'speed': 5}, {'layer': '650mb', 'direction': 250, 'speed': 5}, {'layer': '600mb', 'direction': 285, 'speed': 5}, {'layer': '550mb', 'direction': 280, 'speed': 6}, {'layer': '500mb', 'direction': 270, 'speed': 6}, {'layer': '450mb', 'direction': 275, 'speed': 6}, {'layer': '400mb', 'direction': 285, 'speed': 7}, {'layer': '350mb', 'direction': 280, 'speed': 7}, {'layer': '300mb', 'direction': 290, 'speed': 7}, {'layer': '250mb', 'direction': 295, 'speed': 7}, {'layer': '200mb', 'direction': 305, 'speed': 7}]"

In [9]:
import ast

In [10]:
# apply ast para covertir los valores de la columna wind en listas
df_etl1['wind_profile']= df_etl1['wind_profile'].apply(ast.literal_eval)
df_etl1.head(2)

,Unnamed: 0,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,USA
1,1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,8,1014,none,0,0,180,3,USA


In [11]:
# si hacemos apply sobre una columna que son listas (o dicc) nos genera una columna
# con esos valores : col = key o elemento de la lista 
x = df_etl1['wind_profile'].apply(pd.Series)
x.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 305, 'speed': 3}","{'layer': '850mb', 'direction': 295, 'speed': 3}","{'layer': '800mb', 'direction': 275, 'speed': 5}","{'layer': '750mb', 'direction': 285, 'speed': 5}","{'layer': '700mb', 'direction': 300, 'speed': 6}","{'layer': '650mb', 'direction': 285, 'speed': 6}","{'layer': '600mb', 'direction': 275, 'speed': 5}","{'layer': '550mb', 'direction': 285, 'speed': 6}","{'layer': '500mb', 'direction': 295, 'speed': 7}","{'layer': '450mb', 'direction': 295, 'speed': 7}","{'layer': '400mb', 'direction': 305, 'speed': 6}","{'layer': '350mb', 'direction': 320, 'speed': 9}","{'layer': '300mb', 'direction': 315, 'speed': 10}","{'layer': '250mb', 'direction': 315, 'speed': 12}","{'layer': '200mb', 'direction': 315, 'speed': 7}"
1,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 195, 'speed': 3}","{'layer': '850mb', 'direction': 205, 'speed': 4}","{'layer': '800mb', 'direction': 220, 'speed': 4}","{'layer': '750mb', 'direction': 235, 'speed': 4}","{'layer': '700mb', 'direction': 250, 'speed': 5}","{'layer': '650mb', 'direction': 260, 'speed': 5}","{'layer': '600mb', 'direction': 300, 'speed': 6}","{'layer': '550mb', 'direction': 295, 'speed': 6}","{'layer': '500mb', 'direction': 285, 'speed': 6}","{'layer': '450mb', 'direction': 290, 'speed': 6}","{'layer': '400mb', 'direction': 295, 'speed': 6}","{'layer': '350mb', 'direction': 300, 'speed': 7}","{'layer': '300mb', 'direction': 300, 'speed': 9}","{'layer': '250mb', 'direction': 310, 'speed': 10}","{'layer': '200mb', 'direction': 315, 'speed': 7}"


In [12]:
# VISUALIZAMOS EL RESULTADO
x[0].apply(pd.Series)

,layer,direction,speed
0,950mb,-9999,-9999
1,950mb,-9999,-9999
2,950mb,-9999,-9999
3,950mb,-9999,-9999
4,950mb,-9999,-9999
...,...,...,...
315,950mb,-9999,-9999
316,950mb,-9999,-9999
317,950mb,-9999,-9999
318,950mb,-9999,-9999


In [14]:
# INFO LISTA 'DESEMPAQUETADA' EN DISTINTAS COLUMNAS, AHORA SERIA 'DESEMPAQUETAR'ArithmeticError
# LA INFO DE LOS DICCIONARIOS EN COLUMNAS (KEYS=NOMBRES COLUMNAS, VALUES=VALORES ELEMENTOS)
# CON APPLY PARA TODO EL DATA FRAME CREADO (X)
# EXTRAEMOS LA KEY 'LAYER' Y LA ALMACENAMOS EN VARIABLE COMO STRING

for i in range(len(x.columns)): 
     nombre = "wind" + str(x[i].apply(pd.Series)["layer"][0]) 
     valores = list(x[i].apply(pd.Series)["direction"] )
   
df_etl1.insert(i, nombre, valores)

In [15]:
# REPETIMOS EL PROCESO PARA 'RH_PROFILE'
df_etl1['rh_profile']= df_etl1['rh_profile'].apply(ast.literal_eval)

In [18]:
df_etl1['rh_profile']

0      [{'layer': '950mb', 'rh': -9999}, {'layer': '9...
1      [{'layer': '950mb', 'rh': -9999}, {'layer': '9...
2      [{'layer': '950mb', 'rh': -9999}, {'layer': '9...
3      [{'layer': '950mb', 'rh': -9999}, {'layer': '9...
4      [{'layer': '950mb', 'rh': -9999}, {'layer': '9...
                             ...                        
315    [{'layer': '950mb', 'rh': -9999}, {'layer': '9...
316    [{'layer': '950mb', 'rh': -9999}, {'layer': '9...
317    [{'layer': '950mb', 'rh': -9999}, {'layer': '9...
318    [{'layer': '950mb', 'rh': -9999}, {'layer': '9...
319    [{'layer': '950mb', 'rh': -9999}, {'layer': '9...
Name: rh_profile, Length: 320, dtype: object

In [21]:
y = df_etl1['rh_profile'].apply(pd.Series) 


In [22]:

for i in range(len(y.columns)): 
    nombre = "rh" + str(y[i].apply(pd.Series)["layer"][0]) 
    valores = list(y[i].apply(pd.Series)["rh"] )
  
df_etl1.insert(i, nombre, valores)


In [24]:
# AGRUPAMOS (EN LA TABLA SACAMOS MEDIA DE LAS MEDIDAS CLIMATICAS POR PAÍS)
df_grupo = df_etl1.groupby("pais")[df_etl1.columns].mean().reset_index()

In [25]:
df_grupo.to_csv('etl2.csv')

In [26]:
df_grupo.head(2)

,pais,Unnamed: 0,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,rh200mb,wind200mb,wind10m.direction,wind10m.speed
0,Australia,31.5,97.5,4.468750,-9999.0,-9999.0,-9999.0,32.625000,-1.140625,3.187500,1005.531250,2.640625,0.0,-3.265625,265.390625,93.515625,2.734375
1,New Zealand,31.5,97.5,6.015625,-9999.0,-9999.0,-9999.0,15.046875,5.078125,11.109375,1015.578125,2.921875,0.0,-1.390625,218.437500,143.203125,1.953125
